In [1]:
# Import necessary libraries
import asyncio
import sys
import os
import warnings
from devtools import pprint
# Add the src directory to the Python path and suppress warnings
sys.path.append(os.path.abspath('../../..'))
warnings.filterwarnings("ignore", category=FutureWarning)


# Import MCP components
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# Set up the model

#model = ChatOpenAI(model="gpt-4o")
from dotenv import load_dotenv
load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.getenv("GEMINI_API_KEY"))

from devtools import pprint


In [2]:
from src.agent.market_intelligence_agent.tools.mcp_server import mcp_connection_manager

In [14]:
async def main(query):  # Define an async function to use await
    connection_manager = await mcp_connection_manager()  # Get the manager
    tools = await connection_manager.get_tools()
    pprint(tools)# Await the coroutine
    agent = create_react_agent(model, tools)
    response = await agent.ainvoke({"messages": query})
    return response


In [15]:
query = 'what is the latest news about tarrify between US and China? limit you query within recent 3 days'
response = asyncio.create_task(main(query))

I0000 00:00:1744860294.565364 5609055 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1744860294.791586 5609055 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


[
    StructuredTool(
        name='search',
        description=(
            '\n'
            '    Search the web using Tavily API.\n'
            '    \n'
            '    Args:\n'
            '        query: The search query\n'
            "        search_depth: Depth of search, 'basic' or 'advanced'\n"
            '        topic: Filter results by by either "general" or "news", you should always use "general" unless yo'
            'u are compiled to search for news\n'
            '        days: Number of days to look back for results\n'
            '        time_range: Time range for results, only accepts "day", "week", "month", "year"\n'
            '        max_results: Maximum number of results to return\n'
            '    \n'
            '    Returns:\n'
            '        A dictionary containing search results\n'
            '    '
        ),
        args_schema={
            'properties': {
                'query': {
                    'title': 'Query',
                

In [11]:
pprint(response.result())

<AddableValuesDict({
    'messages': [
        HumanMessage(
            content='what is the latest news about tarrify between US and China? limit you query within recent 3 days',
            additional_kwargs={},
            response_metadata={},
            id='143bdfd6-9a3b-4f1d-aff2-d9c988fff72b',
        ),
        AIMessage(
            content='',
            additional_kwargs={
                'function_call': {
                    'name': 'search',
                    'arguments': '{"query": "tariffs between US and China", "topic": "news", "days": 3.0}',
                },
            },
            response_metadata={
                'prompt_feedback': {
                    'block_reason': 0,
                    'safety_ratings': [],
                },
                'finish_reason': 'STOP',
                'model_name': 'gemini-2.0-flash',
                'safety_ratings': [],
            },
            id='run-3c702454-9feb-4064-b3ed-392d986764e7-0',
            tool_call

In [7]:
print(response.result()['messages'][-1].content)

In the past 3 days, China has raised tariffs on US goods to 125% in response to the US's latest tariff hikes. The US has reportedly imposed tariffs on China as high as 245% on certain imports, while the White House states the total US tariff on China is actually 145%.
